<a href="https://colab.research.google.com/github/sota1111/DL_Zero/blob/main/DL_Zero2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os, sys
os.chdir('/content/drive/MyDrive/DL_Zero')

Mounted at /content/drive


In [6]:
!git config --global user.email "sota.moro@gmail.com"
!git config --global user.name "sota1111"

In [3]:
import numpy as np

# 2章 自然言語と単語の分散表現

## 2.1 自然言語処理とは
自然言語処理：人の言葉をコンピュータに理解させるための技術  
同じ意味の文章でも様々な表現が可能であったり、文章に曖昧さがあったりと、柔軟に意味や形が変わる柔らかさを持つ。  

### 2.1.1 単語の意味
単語：意味の最小単位
単語の意味の表現方法
- シソーラスによる手法
- カウントベースの手法
- 推論ベースの手法

## 2.2 シソーラス
シソーラス：類語辞書  
類義語を上位概念や下位概念で関連付ける。  
motor vehicle->car->SUV, compact, hatch-back

### 2.2.1 WordNet
WordNet：最も有名なシソーラス  
類義語を取得したり、単語ネットワークを利用したりすることができる。単語ネットワークを使って単語間の類似度を算出可能

### 2.2.2 シソーラスの問題点
- 時代の変化に対応するのが困難
- 人の作業コストが高い
- 単語の細かなニュアンスを表現できない

## 2.3 カウントベースの手法
コーパス：大量のテキストデータ  
（自然言語処理の研究やアプリケーションのために目的を持って収集されたテキストデータ）  

### 2.3.1 Pythonによるコーパスの下準備
テキストデータを単語に分割し、その分割した単語を単語IDのリストへと変換する。

In [4]:
def preprocess(text):
  text = text.lower()
  text = text.replace('.', ' .')
  words = text.split(' ')

  word_to_id = {}
  id_to_word = {}
  for word in words:
    if word not in word_to_id:
      new_id = len(word_to_id)
      word_to_id[word] = new_id
      id_to_word[new_id] = word

  corpus = np.array([word_to_id[w] for w in words])

  return corpus, word_to_id, id_to_word

### 2.3.2 単語の分散表現
単語の分散表現：単語をベクトル表現

### 2.3.3 分布仮説
分布仮説：単語の意味は、周囲の単語によって形成される  
コンテキスト：周囲に存在する単語  
ウィンドウサイズ：コンテキストに前後の単語をいくつ含めるか  

### 2.3.4 共起行列
共起行列：共起する単語をテーブルにまとめたもの


In [7]:
!pip install common

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for common: filename=common-0.1.2-py3-none-any.whl size=3732 sha256=ed9a9367861f90482adba48ab75948f61a41da61404a7fa507d7f91105210c8c
  Stored in directory: /root/.cache/pip/wheels/11/88/ea/416ddc295a285f3661c88772a64b4a3766a92ce55f7b01dd9d
Successfully built common


In [9]:
import sys
sys.path.append('..')
import numpy as np

text = 'You say goodbye and I say hello'
corpus, word_to_id, id_to_word = preprocess(text)

print(corpus)

print(id_to_word)

[0 1 2 3 4 1 5]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello'}
